# This Notebook enables you to call every Function in the MTurk Functions catalog

### Prerequisites:
1. You have [an MTurk Requester account and linked AWS account](https://medium.com/@mechanicalturk/setting-up-your-amazon-mechanical-turk-mturk-requester-account-and-aws-account-a316e2f6a156)
2. You have [completed the account set up steps to call AmazonMechanicalTurk and AmazonMechanicalTurkCrowd](https://medium.com/@mechanicalturk/setting-up-your-amazon-mechanical-turk-and-aws-accounts-to-call-a-preview-api-59ade8beafd1)
3. You have configured a local AWS profile to call MTurk with the user you set up in step 2
4. You have [prepaid for HITs in your MTurk Requester account](https://requester.mturk.com/prepayments/new) to cover paying for Worker rewards (suggested amount to start is $5.00)

### If you haven't done so, install the mturk-crowd-beta Python client

In [ ]:
!pip install --upgrade mturk-crowd-beta-client --ignore-installed six

## Next we will import the packages we need to use the API
You'll need to do this before calling any API

In [ ]:
from mturk_crowd_beta_client import MTurkCrowdClient
from boto3.session import Session
import uuid

## Next we set up a session

This examples assume you have a local AWS profile named __'mturk-crowd-caller'__, but you can authenticate however you like, including by directly passing in your access key and secret key.  The access key and secret key are for the User you created during setup with the policies AmazonMechanicalTurkFullAccess and AmazonMechanicalTurkCrowdFullAccess.

In [ ]:
session = Session(profile_name='mturk-crowd-caller')

crowd_client = MTurkCrowdClient(session)

## Available APIs:

* [sentiment-analysis](#sentiment-analysis)
* [image-contains](#image-contains)
* [emotion-detection](#emotion-detection)
* [semantic-similarity](#semantic-similarity)
* [collect-utterance-text](#collect-utterance-text)
* [image-categorization](#image-categorization)
* [text-intent-detection](#text-intent-detection)
* [text-categorization](#text-categorization)
* [named-entity-recognition](#named-entity-recognition)
* [bounding-box](#bounding-box)
* [image-similarity](#image-similarity)
* [co-reference-resolution](#co-reference-resolution)
* [key-phrase-extraction](#key-phrase-extraction)

## <a id='sentiment-analysis'>sentiment-analysis</a>

Input:
{
 "text": "Everything is wonderful!"
}

Result: {'sentiment': 'positive'}

Max length of the input text is 400 characters.  Sentiment is one of positive, negative, neutral or cannot determine.  We ask up to 5 Workers until 2 of them agree on the answer.


When you create a Task using the sentiment-analysis API, you're automatically creating a Human Intelligence Task (HIT) on worker.mturk.com.  Here's an example of a sentiment analysis HIT.

![sentiment-analysis-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/sentiment-analysis-HIT-example.png)

### Create a Task

In [ ]:
#set the function_name to the name of the API
function_name = 'sentiment-analysis'

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text = 'The trip by @VP Pence was long planned. He is receiving great praise for leaving game after the players showed such disrespect for country!'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

### Creating multiple Tasks and Processing Results 

Next, we'll read from a CSV file that has multiple rows of text that needs to be analyzed for sentiment using the pandas library.  As a prerequisite you need to have installed the pandas library.

In [ ]:
import pandas as pd

In [ ]:
#read the input data from csv into a pandas DataFrame
data = pd.read_csv('https://s3-us-west-2.amazonaws.com/mturk-sample-datasets/sentiment-analysis-example-inputs.csv')

In [ ]:
# column 1 is the Task ID and column2 is the input data
data

In [ ]:
#loop through the rows and create a Task per row, write the put_task status in a new column
status_codes = []
for index, row in data.iterrows():
    put_result = crowd_client.put_task(function_name, row.task_id,{'text': row.review_text})
    status_codes.append(put_result.status_code)
data['status_codes'] = status_codes

In [ ]:
data

We'll call get_task periodically until all of the Tasks reach the "completed" state.

In [ ]:
# loop through the rows and get results, store the state and results in new columns

status_codes = []
responses = []
for index, row in data.iterrows():
    get_result = crowd_client.get_task(function_name, row.task_id)
    status_codes.append(get_result.status_code)
    responses.append(get_result.json())

In [ ]:
data['status_codes'] = status_codes
data['state'] = [r['state'] for r in responses]
data['problem_details'] = [r['problemDetails'] for r in responses]
#need to handle the case when not all tasks are in completed state
data['sentiment'] = [r['result']['sentiment'] for r in responses]

In [ ]:
data

## <a id='image-contains'>image-contains</a>

Given an image and a type of object to look for, this API returns true or false.

Input: `{"image": {"url": "https://www.mturk.com/media/butterbean.jpg"}, "target": {"label": "dog"} }`

Result: `{"containsTarget": true}`

Internal Only: we ask 2 - 5 Workers to get agreement.

In [ ]:
function_name = 'image-contains'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# the URL of the image that you want annotated
image_url = 'https://urbanedge.blogs.rice.edu/files/2016/02/midtown-15wd4ck.jpg'

### The type of thing we're looking for

In [ ]:
label = 'Pedestrians'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                                   task_name,
                                   {'image': {'url': image_url}, 'target': {'label': label}})

print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))        

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(
{'status_code': get_result.status_code, 'task': get_result.json()}))

### Creating multiple Tasks and Processing Results 

Next, we'll read from a CSV file that has multiple rows of text that needs to be analyzed for sentiment using the pandas library.  As a prerequisite you need to have installed the pandas library.

In [ ]:
import pandas as pd

In [ ]:
#read the input data from csv into a pandas DataFrame
data = pd.read_csv('https://s3-us-west-2.amazonaws.com/mturk-sample-datasets/image-contains-example-inputs.csv')

In [ ]:
# column 1 is the Task ID and column2 amd 3 are the input data
data

In [ ]:
#loop through the rows and create a Task per row, write the put_task status in a new column
status_codes = []
for index, row in data.iterrows():
    put_result = crowd_client.put_task(function_name, 
                                       row.task_id,
                                       {'image': {'url': row.image_url},
                                        'target': {'label': row.Label} })
    status_codes.append(put_result.status_code)
data['status_codes'] = status_codes

In [ ]:
data

We'll call get_task periodically until all of the Tasks reach the "completed" state.

In [ ]:
# loop through the rows and get results, store the state and results in new columns

status_codes = []
responses = []
for index, row in data.iterrows():
    get_result = crowd_client.get_task(function_name, row.task_id)
    status_codes.append(get_result.status_code)
    responses.append(get_result.json())

In [ ]:
data['status_codes'] = status_codes
data['state'] = [r['state'] for r in responses]
data['problem_details'] = [r['problemDetails'] for r in responses]
#need to handle the case when not all tasks are in completed state
data['containsTarget'] = [r['result']['containsTarget'] for r in responses]

In [ ]:
data

## <a id='emotion-detection'>emotion-detection</a>

This API determines the emotion of text.

Input:{"text": "First time ever winning all three fantasy leagues AND @Seahawks win!"}
Result: {"emotion": "joy"}

Emotion is one of Joy, Anger, Fear, Sadness, Surprise, Disgust, or Neutral.  Internal only: we ask up to 9 Workers to get agreement.

![emotion-detection-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/emotion-detection-HIT-example.png)

In [ ]:
function_name = 'emotion-detection'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text = 'Just realised that I have #280characters so now I can finally tweet this: "Daenerys Stormborn of the House Targaryen, First of Her Name, the Unburnt, Queen of the Andals and the First Men, Khaleesi of the Great Grass Sea, Breaker of Chains, and Mother of Dragons'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id=semantic-similarity>semantic-similarity</a>

Compare two text documents and rate them on how similar they are, on a scale between 0 and 1 where 1 is very similar.

Input: {"text1": "The sky is blue.", "text2": "The sky was the color of blue."}

Result: {"similarityScore": 0.75}

In [ ]:
function_name = 'semantic-similarity'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text1 = 'I\'m so hungry I could eat a horse'
text2 = 'I\'m hangry'

In [ ]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text1': text1, 'text2': text2})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

### Creating multiple Tasks and Processing Results 

Next, we'll read from a CSV file that has multiple rows of text that needs to be analyzed for sentiment using the pandas library.  As a prerequisite you need to have installed the pandas library.

In [ ]:
import pandas as pd

In [ ]:
#read the input data from csv into a pandas DataFrame
data = pd.read_csv('https://s3-us-west-2.amazonaws.com/mturk-sample-datasets/semantic-similarity-example-inputs.csv')

In [ ]:
# column 1 is the Task ID and column2 is the input data
data

In [ ]:
#loop through the rows and create a Task per row, write the put_task status in a new column
status_codes = []
for index, row in data.iterrows():
    put_result = crowd_client.put_task(function_name, 
                                       row.task_id,
                                        {'text1': row.review1, 'text2': row.review2})
    status_codes.append(put_result.status_code)
data['status_codes'] = status_codes

In [ ]:
data

We'll call get_task periodically until all of the Tasks reach the "completed" state.

In [ ]:
# loop through the rows and get results, store the state and results in new columns

status_codes = []
responses = []
for index, row in data.iterrows():
    get_result = crowd_client.get_task(function_name, row.task_id)
    status_codes.append(get_result.status_code)
    responses.append(get_result.json())

In [ ]:
data['status_codes'] = status_codes
data['state'] = [r['state'] for r in responses]
data['problem_details'] = [r['problemDetails'] for r in responses]
#need to handle the case when not all tasks are in completed state
data['similarityScore'] = [r['result']['similarityScore'] for r in responses]

In [ ]:
data

## <a id='collect-utterance-text'>collect-utterance-text</a>

Given a context and an intention, provide what you would say, in text, in that situation.

input: {"context": "Someone recently bought a phone and it doesn't work", "intent": "They want to return a phone"}'

result: {'utterances': [{'text': "result: {"utterance": "The phone that I just bought stopped working.  I want to get a refund"}
"}]}

Internal only: We ask only 1 Worker for each task.

Here's an example of what the HIT might look like to Workers:
![collect-utterance-text-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/collect-utterance-text-HIT-example.png)

In [ ]:
function_name = 'collect-utterance-text'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# provide the context and the intent
context = 'Someone is calling their doctor\'s office'
intent = 'I want to change an existing appointment'

In [ ]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'context': context, 'intent': intent})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id='image-categorization'>image-categorization</a>

Given an image URL and a list of categories, determine which category the image best belongs to.  You can optionally provide a description for each category to provide more guidance on what each category includes.

Input: {"image":{"url": "https://requester.mturk.com/assets/simon.jpg"},"categories": “categories”: [{“label”: “Plant”}, {“label”:”Animal”}, {“label”:”Bacteria”}, {“label”:”Fungi”}, {“label”:”Protists”}]

Result: {"category":"Animal"} 

![image-categorization-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/image-categorization-HIT-example.png)

In [ ]:
function_name = 'image-categorization'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# provide the context and the intent
image_url = 'https://www.abebooks.com/images/books/harry-potter/sorcerers-stone.jpg'
categories =  [{'label': 'Hardcover', 'description': 'hardcover version'}, {'label':'Paperback', 'description': 'paperback version'}, {'label':'eBook', 'description': 'digital text version, including PDF, Kindle'}, {'label':'Audio Book', 'description': 'any audio version, including CD, mp3, streaming'}]

In [ ]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                            {'image': {'url': image_url},
                            'categories': categories})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id='text-intent-detection'>text-intent-detection</a>

Given a list of intentions, categorize the best intention that matches the text given.

input: {"text": "foo", "intents": [{"label": "Schedule an appointment", "description": "example: I need to make an appointment with Dr. Smith"}, {"label": "Medical Record Request", "description": "I need a copy of my kids&#39; immunication records"}]}

resut:{'applicableIntents': [{'label': 'Schedule an appointment'}]}

Here's an example of what the task might look like for Workers.
![text-intent-detection-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/text-intent-detection-HIT-example.png)

In [ ]:
function_name = 'text-intent-detection'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text to be analysed and a list of intentions to categorize the text for
text = 'my son was stung by bees and I need to know if I need to go to the ER'
intents =  [{'label': 'Schedule an appointment', 'description': 'e.g. I need to make an appointment with Dr. Smith'}, {'label': 'Medical Record Request', 'description': 'e.g. I need a copy of my kids immunization records'}]

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                                   task_name,
                                  {'text': text,
                                  'intents': intents})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id=text-categorization>text-categorization</a>

Given a piece of text and a list of categories, choose the best cateogry that fits the text.

input: {"text": "The Baltimore Ravens beat the Seattle Seahawks 24-7", "categories": [{"label": "sports", "description": "talks about sports"}, {"label": "food", "description": "talks about food"}]}

result: {'applicableCategories': [{'label': 'sports'}]}

Here's an example of a HIT to MTurk Workers generated by calling this API
![text-categorization-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/text-categorization-HIT-example.png)

In [ ]:
function_name = 'text-categorization'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text to be analysed and a list of categories to assess for
text = 'These are great. They do run a touch small. I almost could go a half size up from my normal size.'
categories =  [{'label': 'style', 'description': 'related to the look of the product'}, {'label': 'fit', 'description': 'related to the sizing or how it fits'}, {'label': 'quality', 'description': 'related to how well made the product is'}, {'label': 'price', 'description': 'related to cost or value of the product'}]

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                                   task_name,
                                  {'text': text,
                                  'categories': categories})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id=named-entity-recognition>named-entity-recognition</a>

Given a paragraph of text, label the span of text that represent entities of various categories, such as person and location.

input: {'text': 'Harry Potter is midway through his training as a wizard and his coming of age. Harry wants to get away from the pernicious Dursleys and go to the International Quidditch Cup. He wants to find out about the mysterious event that's supposed to take place at Hogwarts this year, an event involving two other rival schools of magic, and a competition that hasn't happened for a hundred years. He wants to be a normal, fourteen-year-old wizard. But unfortunately for Harry Potter, he's not normal - even by wizarding standards. And in his case, different can be deadly.'}

result: {'entities': [{'end': 12, 'start': 0, 'text': 'Harry Potter', 'type': 'person'}, {'end': 84, 'start': 79, 'text': 'Harry', 'type': 'person'}, {'end': 131, 'start': 123, 'text': 'Dursleys', 'type': 'person'}, {'end': 77, 'start': 60, 'text': 'his coming of age', 'type': 'date'}, {'end': 173, 'start': 146, 'text': 'International Quidditch Cup', 'type': 'event'}, {'end': 222, 'start': 217, 'text': 'event', 'type': 'event'}, {'end': 284, 'start': 279, 'text': 'event', 'type': 'event'}, {'end': 346, 'start': 335, 'text': 'competition', 'type': 'event'}, {'end': 264, 'start': 256, 'text': 'Hogwarts', 'type': 'organization'}, {'end': 318, 'start': 311, 'text': 'schools', 'type': 'organization'}, {'end': 264, 'start': 256, 'text': 'Hogwarts', 'type': 'organization'}]}

![NER-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/ner-HIT-example.png)

### Create a Task

In [ ]:
#set the function_name to the name of the API
function_name = 'named-entity-recognition'

In [ ]:
# automatically generate a random task ID
# task_name = 'my-test-task-' + uuid.uuid4().hex
task_name='my-test-task-ff7d0b50791a43cc819b60b7b568c422'
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text = 'Harry Potter is midway through his training as a wizard and his coming of age. Harry wants to get away from the pernicious Dursleys and go to the International Quidditch Cup. He wants to find out about the mysterious event that\'s supposed to take place at Hogwarts this year, an event involving two other rival schools of magic, and a competition that hasn\'t happened for a hundred years.'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id='bounding-box'> bounding-box </a>

Given an image, draw a box around particular objects, specified by labels.  This API returns the coordinates of labeled objects in the image.

input: {"image": {"url": "https://sierra-videos.s3.amazonaws.com/2017_0521/DJI_0001_frames_720h/1495411174030000000.jpg"}, "labels": [{"label": "car"}, {"label": "pedestrians"}]}

result: {'labels': [{'boundingBox': {'height': 55, 'left': 896, 'top': 665, 'width': 51}, 'label': 'car'}, {'boundingBox': {'height': 50, 'left': 555, 'top': 515, 'width': 63}, 'label': 'car'}, {'boundingBox': {'height': 51, 'left': 539, 'top': 590, 'width': 58}, 'label': 'car'}, {'boundingBox': {'height': 44, 'left': 216, 'top': 512, 'width': 55}, 'label': 'car'}, {'boundingBox': {'height': 54, 'left': 873, 'top': 475, 'width': 53}, 'label': 'car'}, {'boundingBox': {'height': 56, 'left': 1083, 'top': 490, 'width': 47}, 'label': 'car'}, {'boundingBox': {'height': 42, 'left': 164, 'top': 659, 'width': 66}, 'label': 'car'}, {'boundingBox': {'height': 46, 'left': 524, 'top': 213, 'width': 50}, 'label': 'car'}, {'boundingBox': {'height': 48, 'left': 814, 'top': 311, 'width': 53}, 'label': 'car'}]}

This API currently only supports upright bounding boxes.  

Here's an example of what the Worker sees:
![bound-box-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/bound-box-HIT-example.png)

In [ ]:
function_name = 'bounding-box'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# provide the context and the intent
image_url = 'https://sierra-videos.s3.amazonaws.com/2017_0521/DJI_0001_frames_720h/1495411174030000000.jpg'
labels =  [{"label": "car"}, {"label": "pedestrians"}]

In [ ]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                            {'image': {'url': image_url},
                            'labels': labels})

print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id=image-similarity>image-similarity</a>
This API takes in two images, specified by URL, and returns a score for how similar those two images are.  

input: {'input': {'image1': {'url': 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1a.png'}, 'image2': {'url': 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1b.png'}}

result: {'similarityScore': 0.75}

Here's an example of a HIT that's shown to Workers:
![image-similarity-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/image-similarity-HIT-example.png)

In [ ]:
function_name = 'image-similarity'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# the URLs of the images that you want to compare
image_url_1 = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1a.png'
image_url_2 = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1b.png'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,task_name,
                                   {'image1': {'url': image_url_1},
                                    'image2': {'url': image_url_2}})

print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))        

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id=co-reference-resolution>co-reference-resolution</a>

Co-reference resolution is the task of finding all expressions that refer to the same entity in a text. This API returns references and the position of the start and end of each reference.

input: { "text": "Megan said "I voted for Hilary because she better aligns with my values"" }

result:  {'references': {'ref-1': [{'end': 6, 'start': 0, 'text': 'Megan ', 'type': 'ref-1'}, {'end': 14, 'start': 12, 'text': 'I ', 'type': 'ref-1'}, {'end': 64, 'start': 62, 'text': 'my', 'type': 'ref-1'}], 'ref-2': [{'end': 30, 'start': 24, 'text': 'Hilary', 'type': 'ref-2'}, {'end': 42, 'start': 39, 'text': 'she', 'type': 'ref-2'}]}}

Internal only: this API currently only asks one Worker to complete the HIT, excluding specific Workers who have been put on an exclusion list because of poor past performance on NLP related HITs.

Here's an example of a HIT that's shown to Workers:
![co-reference-resolution-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/co-reference-resolution-HIT-example.png)

### Create a Task

In [ ]:
#set the function_name to the name of the API
function_name = 'co-reference-resolution'

In [ ]:
# automatically generate a random task ID
#task_name = 'my-test-task-' + uuid.uuid4().hex
task_name = 'my-test-task-656e8384878a40d59f55289a48085d0d'
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text = 'Megan said \"I voted for Hilary because she better aligns with my values\"'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id=keyphrase-extraction>key-phrase-extraction</a> 

Key phrases provide a concise description of a document’s content; they are useful for document categorization, clustering, indexing, search, and summarization; quantifying semantic similarity with other documents; as well as conceptualizing particular knowledge domains.  Key phrase extraction is the process of extracting the important and topic words in a document.

input: { "text": "This is the second Boots I brought from this brand, the material they are used to make these boots are quite decent, it’s not real leather though, but once you get it on hand you can tell that it is definitely high quality material, the last boots I brought cost me 45 bucks, it lasted about 1 year and half, very remarkable durability I got to say, beside this the interior is super soft and very comfortable to wear. If you are looking for mid-range priced boots with high quality this is the best shot for you." }

result: {'keyPhrases': [{'end': 25, 'start': 19, 'text': 'Boots '}, {'end': 115, 'start': 103, 'text': 'quite decent'}, {'end': 138, 'start': 122, 'text': 'not real leather'}, {'end': 231, 'start': 210, 'text': 'high quality material'}, {'end': 307, 'start': 292, 'text': '1 year and half'}, {'end': 335, 'start': 309, 'text': 'very remarkable durability'}, {'end': 388, 'start': 378, 'text': 'super soft'}, {'end': 409, 'start': 393, 'text': 'very comfortable'}, {'end': 458, 'start': 442, 'text': 'mid-range priced'}, {'end': 482, 'start': 470, 'text': 'high quality'}]}

Internal only: this API currently only asks one Worker to complete the HIT, excluding specific Workers who have been put on an exclusion list because of poor past performance on NLP related HITs.

Here's an example of a HIT that's shown to Workers: 
![key-phrase-extraction-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/key-phrase-extraction-HIT-example.png)

### Create a Task

In [ ]:
#set the function_name to the name of the API
function_name = 'key-phrase-extraction'

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text = 'This is the second Boots I brought from this brand, the material they are used to make these boots are quite decent, it’s not real leather though, but once you get it on hand you can tell that it is definitely high quality material, the last boots I brought cost me 45 bucks, it lasted about 1 year and half, very remarkable durability I got to say, beside this the interior is super soft and very comfortable to wear. If you are looking for mid-range priced boots with high quality this is the best shot for you.'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))